In [1]:
from Dataset import Dataset
from conditonalDCGAN import ConditionalGenerator, ConditionalDiscriminator
from DCGAN import Generator, Discriminator
from model_wrapper import Trainer
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import os
import shutil


In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
GAN_LOGS = os.path.join(os.getcwd(), "tboard_logs", "gan")
if not os.path.exists(GAN_LOGS):
    os.makedirs(GAN_LOGS)

shutil.rmtree(GAN_LOGS)
writer = SummaryWriter(GAN_LOGS)


# Create a dataset and a DataLoader
transforms = {'val':transforms.Compose([
    transforms.Resize([32, 32]),  # Resize the image first
    transforms.ToTensor(),        # Convert PIL image to tensor
]),
'train':transforms.Compose([
    transforms.Resize([32, 32]),  # Resize the image first
    transforms.RandomHorizontalFlip(p=0.5),       # Random horizontal flip
    transforms.ToTensor(),        # Convert PIL image to tensor
])
}

In [4]:
root_dir = "/home/user/zafara1/Cuda_Vision_Lab-/Session5/Solution/data/afhq"

# Initialize dataset
train_dataset = Dataset(root_dir=root_dir, split="train", transform=transforms["train"])
test_dataset = Dataset(root_dir=root_dir, split="val", transform=transforms["val"])


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=8,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,num_workers=8,pin_memory=True)
print("test_dataset per epoch",next(iter(test_loader))[0].size())


test_dataset per epoch torch.Size([64, 3, 32, 32])


In [ ]:
# Initialize models, loss function, and optimizers
generator = Generator(latent_dim=128, num_channels=3, base_channels=32, num_classes=3).to("cuda:0")
discriminator = Discriminator(in_channels=3, out_dim=1, base_channels=32, num_classes=3).to("cuda:0")


optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [10]:
trainer = Trainer(generator=generator, discriminator=discriminator, latent_dim=128, conditional_gan=False,writer=writer)

In [12]:
trainer.train(data_loader=train_loader)

  0%|          | 0/229 [00:00<?, ?it/s]

  0%|          | 0/229 [00:00<?, ?it/s]


TypeError: Discriminator.forward() missing 1 required positional argument: 'labels'

In [5]:
# Initialize models, loss function, and optimizers
conditional_generator = ConditionalGenerator(latent_dim=128, num_channels=3, base_channels=32, num_classes=3).to("cuda:0")
conditional_discriminator = ConditionalDiscriminator(in_channels=3, out_dim=1, base_channels=32, num_classes=3).to("cuda:0")

optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [8]:
conditionaltrainer = Trainer(generator=conditional_generator, discriminator=conditional_discriminator, latent_dim=128, conditional_gan=True,writer=writer)

In [ ]:
conditionaltrainer.train(data_loader=train_loader)

  0%|          | 0/229 [00:00<?, ?it/s]

Epoch 5 Iter 1045: D_Loss=0.0, G_Loss=34.51628):  57%|█████▋    | 130/229 [00:03<00:02, 33.88it/s]


KeyboardInterrupt: 